In [18]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import csv
import os
import seaborn as sns
import sklearn
import joblib

from datetime import datetime
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

print("Pandas version: ", pd.__version__)
print("Numpy version: ", np.__version__)
print("Matplotlib.pyplot version: ", matplotlib.__version__)
print("Seaborn version: ", sns.__version__)
print("SciKitLearn version: ", sklearn.__version__)

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/Users/melanievollmar/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Users/melanievollmar/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Users/melanievollmar/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Users/melanievollmar/opt/anaconda3/lib/python3.8/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Users/melanievollmar/opt/anaconda3/lib/python3.8/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: dlopen(/Users/melanievollmar/opt/anaconda3/lib/pytho

In [17]:
!pip3 install tensorflow


In [3]:
# Find the input data as CSV
run1_3dii_csv = "/Users/melanievollmar/Documents/METRICS/data_analysis_20210312/BigEP_samples/summary_table_3dii_allData_allXtals_run1_BigEPruns.csv"
run2_3dii_csv = "/Users/melanievollmar/Documents/METRICS/data_analysis_20210312/BigEP_samples/summary_table_3dii_allData_allXtals_run2_BigEPruns.csv"
run3_3dii_csv = "/Users/melanievollmar/Documents/METRICS/data_analysis_20210312/BigEP_samples/summary_table_3dii_allData_allXtals_run3_BigEPruns.csv"
run4_3dii_csv = "/Users/melanievollmar/Documents/METRICS/data_analysis_20210312/BigEP_samples/summary_table_3dii_allData_allXtals_run4_BigEPruns.csv"


run1_dials_csv = "/Users/melanievollmar/Documents/METRICS/data_analysis_20210312/BigEP_samples/summary_table_dials_allData_allXtals_run1_BigEPruns.csv"
run2_dials_csv = "/Users/melanievollmar/Documents/METRICS/data_analysis_20210312/BigEP_samples/summary_table_dials_allData_allXtals_run2_BigEPruns.csv"
run3_dials_csv = "/Users/melanievollmar/Documents/METRICS/data_analysis_20210312/BigEP_samples/summary_table_dials_allData_allXtals_run3_BigEPruns.csv"
run4_dials_csv = "/Users/melanievollmar/Documents/METRICS/data_analysis_20210312/BigEP_samples/summary_table_dials_allData_allXtals_run4_BigEPruns.csv"


# Load the CSV data into a Pandas dataframe
run1_3dii_df = pd.read_csv(run1_3dii_csv)
run2_3dii_df = pd.read_csv(run2_3dii_csv)
run3_3dii_df = pd.read_csv(run3_3dii_csv)
run4_3dii_df = pd.read_csv(run4_3dii_csv)


run1_dials_df = pd.read_csv(run1_dials_csv)
run2_dials_df = pd.read_csv(run2_dials_csv)
run3_dials_df = pd.read_csv(run3_dials_csv)
run4_dials_df = pd.read_csv(run4_dials_csv)


df_3dii = pd.concat([run1_3dii_df, run2_3dii_df, run3_3dii_df, run4_3dii_df],
               ignore_index=True)
df_run234_3dii = pd.concat([run2_3dii_df, run3_3dii_df, run4_3dii_df],
               ignore_index=True)
#print(df_3dii.describe)

dials_df = pd.concat([run1_dials_df, run2_dials_df, run3_dials_df, run4_dials_df],
               ignore_index=True)
dials_run234_df = pd.concat([run2_dials_df, run3_dials_df, run4_dials_df],
               ignore_index=True)
#print(dials_df.describe)


full_3dii_df = df_3dii[['anomalousCC', 'lowreslimit', 'anomalousslope', 'diffF',
       'diffI', 'f', 'autobuild_success', 'crank2_success', 'autosharp_success']]

full_dials_df = dials_df[['anomalousCC', 'lowreslimit', 'anomalousslope', 'diffF',
       'diffI', 'f', 'autobuild_success', 'crank2_success', 'autosharp_success']]



In [4]:
# train-test-split
# this should be done stratified for multi-class multi-lable; the only
# support I found is in http://scikit.ml which is no longer maintained
# (as of 2018); I stick to normal train-test-split for now, also considering
# that in on ecase I have only one sample available; use the standard
# sklearn function and apply stratification

# train-test-split for 3dii
y_3dii = full_3dii_df[['autobuild_success', 'crank2_success', 'autosharp_success']]
X_3dii = full_3dii_df.iloc[:, :-4]

X_3dii_train, X_3dii_test, y_3dii_train, y_3dii_test = train_test_split(X_3dii, y_3dii,
                                                                        test_size=0.2,
                                                                        random_state=42,
                                                                        stratify=y_3dii)
print("Training X shape: ", X_3dii_train.shape)
print("Testing X shape: ", X_3dii_test.shape)
print("Training y shape: ", y_3dii_train.shape)
print("Training y shape: ", y_3dii_test.shape)
print(X_3dii_train[:10], y_3dii_train[:10])

# train-test-split for DIALS
# needed to drop the sample for which there is only one occurance in order
# to have train-test-split run
full_dials_df = full_dials_df.drop(full_dials_df[(full_dials_df['autobuild_success']==0) &
                                                 (full_dials_df['crank2_success']==0) &
                                                 (full_dials_df['autosharp_success']>0)].index)

y_dials = full_dials_df[['autobuild_success', 'crank2_success', 'autosharp_success']]
X_dials = full_dials_df.iloc[:, :-4]

X_dials_train, X_dials_test, y_dials_train, y_dials_test = train_test_split(X_dials, y_dials,
                                                                        test_size=0.2,
                                                                        random_state=42,
                                                                        stratify=y_dials)
print("Training X shape: ", X_dials_train.shape)
print("Testing X shape: ", X_dials_test.shape)
print("Training y shape: ", y_dials_train.shape)
print("Training y shape: ", y_dials_test.shape)
print(X_dials_train[:10], y_dials_train[:10])

Training X shape:  (886, 5)
Testing X shape:  (222, 5)
Training y shape:  (886, 3)
Training y shape:  (222, 3)
      anomalousCC  lowreslimit  anomalousslope    diffF    diffI
364       0.21259     55.50788         1.07108  0.11237  0.97740
371       0.49710     55.62809         1.01347  0.04157  0.92399
807       0.63819     57.91000         1.28575  0.09967  1.36307
926       0.26837     41.01219         1.12231  0.12078  1.03507
898       0.59658     28.94953         1.13920  0.09804  1.20998
638       0.62511     58.05000         1.28433  0.06746  1.48108
952       0.63506     48.57000         1.12615  0.11191  1.10462
1069      0.45966     80.80692         1.18006  0.14088  1.00073
88        0.69392     80.64429         1.14394  0.05382  1.23263
721       0.54550     58.18000         1.32375  0.08737  1.32208       autobuild_success  crank2_success  autosharp_success
364                   0               0                  0
371                   0               0                 

In [6]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # make a prediction on the test set
        yhat = model.predict(X_test)
        # round probabilities to class labels
        yhat = yhat.round()
        # calculate accuracy
        acc = accuracy_score(y_test, yhat)
        # store result
        print('>%.3f' % acc)
        results.append(acc)
    return results


# evaluate model
results = evaluate_model(X_3dii.to_numpy(), y_3dii.to_numpy())
# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

NameError: name 'Sequential' is not defined

In [7]:
def forest_best_params(X_train, y_train):
    '''create a new random forest using the best parameter combination found above'''
    print('*' *80)
    print('*    Building new forest based on best parameter combination and save as pickle')
    print('*' *80)
    
    clf = DecisionTreeClassifier(criterion='entropy',
                                  max_depth=3,
                                  max_features=2,
                                  max_leaf_nodes=17,
                                  min_samples_leaf=8,
                                  min_samples_split=18,
                                  random_state= 0)
    clf_ada = AdaBoostClassifier(clf,
                                 learning_rate=0.6355,
                                 n_estimators=5694,
                                 algorithm ="SAMME.R",
                                 random_state=5)
                                
    clf_ada.fit(X_train, y_train)

    print(clf_ada.estimators_)
    print(clf_ada.feature_importances_)
    
    attr = ['diffI', 'anomalousCC', 'lowreslimit', 'anomalousslope', 'diffF', 'f']
    
    feature_importances_transform = clf_ada.feature_importances_
    feature_importances_transform_ls = sorted(zip(feature_importances_transform, attr), reverse=True)
    print(feature_importances_transform_ls)
    feature_importances_ls = np.mean([tree.feature_importances_ for tree in clf_ada.estimators_], axis=0)

#    def feature_importances_best_estimator(feature_list, name, directory):
#      datestring = datetime.strftime(datetime.now(), '%Y%m%d_%H%M')
#      feature_list.sort(key=lambda x: x[1], reverse=True)
#      feature = list(zip(*feature_list))[1]
#      score = list(zip(*feature_list))[0]
#      x_pos = np.arange(len(feature))
#      plt.bar(x_pos, score,align='center')
#      plt.xticks(x_pos, feature, rotation=90, fontsize=2)
#      plt.title('Histogram of Feature Importances for best Tree using features %s ' %name)
#      plt.xlabel('Features')
#      plt.tight_layout()
#      plt.savefig(os.path.join(directory, 'feature_importances_best_bar_plot_rand_ada_'+name+datestring+'.png'))     
#      plt.close()
#    
#    feature_importances_best_estimator(feature_importances_transform_ls, 'newdata_minusEP', self.newdata_minusEP)
#    
#
#    def feature_importances_pandas(clf, X_train, name, directory):   
#      datestring = datetime.strftime(datetime.now(), '%Y%m%d_%H%M')      
#      feature_list = []
#      for tree in clf.estimators_:
#        feature_importances_ls = tree.feature_importances_
#        feature_list.append(feature_importances_ls)
#        
#      df = pd.DataFrame(feature_list, columns=X_train.columns)
#      df_mean = df[X_train.columns].mean(axis=0)
#      df_std = df[X_train.columns].std(axis=0)
#      df_mean.plot(kind='bar', color='b', yerr=[df_std], align="center", figsize=(20,10), rot=90)
#      plt.title('Histogram of Feature Importances over all trees in AdaBoostClassifier using features %s ' %name)
#      plt.xlabel('Features')
#      plt.tight_layout()
#      plt.savefig(os.path.join(directory, 'feature_importances_overall_bar_plot_rand_ada_'+name+datestring+'.png'))
#      plt.close()
#      
#    feature_importances_pandas(self.tree_clf_rand_ada_new_transform, self.X_newdata_transform_train, 'newdata_minusEP', self.newdata_minusEP)
#
#    def write_pickle(forest, directory, name):
#      datestring = datetime.strftime(datetime.now(), '%Y%m%d_%H%M')
#      joblib.dump(forest, os.path.join(directory,'best_forest_rand_ada_'+name+datestring+'.pkl'))
#      with open(os.path.join(directory, 'decisiontree_ada_randomsearch.txt'), 'a') as text_file:
#        text_file.write('Created new random forest "forest_clf_rand_ada_new_%s" using best parameters \n' %name)
#        text_file.write('Creating pickle file for best forest as best_forest_rand_ada_%s.pkl \n' %name)
#    
#    write_pickle(self.tree_clf_rand_ada_new_transform, self.newdata_minusEP, 'newdata_minusEP')
    
forest_best_params(X_3dii_train, y_3dii_train)
#forest_best_params(X_dials_train, y_dials_train)

********************************************************************************
*    Building new forest based on best parameter combination and save as pickle
********************************************************************************


ValueError: y should be a 1d array, got an array of shape (886, 3) instead.